# Call center case - Post-call transcription and analysis with Azure OpenAI Service

In this exercise, we will perform sentiment analysis and summerization using call center transcriptions. We will transribe the customer recording to text, then use OpenAI to detect sentiment. We also use OpenAI to summerize long text into a few sentences for further analysis.

In [ ]:
!pip install azure.cognitiveservices.speech
!pip install num2words

In [1]:
# from azure.ai.textanalytics import TextAnalyticsClient
# from azure.core.credentials import AzureKeyCredential
import azure.cognitiveservices.speech as speechsdk

import json, os
import string
import time
import wave

import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

openai.api_type = "azure"
openai.api_key = os.getenv('AZURE_OPENAI_API_KEY') 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

openai.api_version = "2022-06-01-preview"

SPEECH_KEY = os.environ["SPEECH_API_KEY"]

COMPLETIONS_MODEL = "text-davinci-003"

def recognize_speech_from_file(filename):
    # Set up the subscription info for the Speech Service:
    # Replace with your own subscription key and service region (e.g., "westus").
    speech_key = SPEECH_KEY
    service_region = "eastus"

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,  audio_config=audio_config)

    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        # print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    # speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    # speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

C:\Users\jeffreylai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Sentiment Analysis
### Transcribe Customer Call to Text

In [2]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)

STT SESSION STARTED: SessionEventArgs(session_id=a01bc6cefea04c3bb15d08ff4c00d936)
STT SESSION STOPPED SessionEventArgs(session_id=a01bc6cefea04c3bb15d08ff4c00d936)
CLOSING on SessionEventArgs(session_id=a01bc6cefea04c3bb15d08ff4c00d936)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Well, come to find out, it did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


### Create Promot for sentiment analysis
Use natural language to instruct OpenAI to detect customer's sentiment

In [3]:
prompt = f"Detect whether customer is positive or negative.  Just say positive or negative.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Positive.'

### Use a negative example

In [4]:
text = recognize_speech_from_file("../data/bad_review.wav")
print(text)
prompt = f"Detect whether customer is positive or negative. Just say positive or negative.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

STT SESSION STARTED: SessionEventArgs(session_id=0c5b7c8b7dbb46a697939421b46f98ea)
STT SESSION STOPPED SessionEventArgs(session_id=0c5b7c8b7dbb46a697939421b46f98ea)
CLOSING on SessionEventArgs(session_id=0c5b7c8b7dbb46a697939421b46f98ea)
['Installed with five kilowatts heater in garage.', 'Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


'Negative'

# Summarization

Use OpenAI to summerize customer message

In [5]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)
prompt = f"Summerize the following text.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

STT SESSION STARTED: SessionEventArgs(session_id=2399e71a9fb7432db81756cc761e92c1)
STT SESSION STOPPED SessionEventArgs(session_id=2399e71a9fb7432db81756cc761e92c1)
CLOSING on SessionEventArgs(session_id=2399e71a9fb7432db81756cc761e92c1)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Well, come to find out, it did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


'This customer purchased a thermostat from a big box store, which failed to turn off when it got too cold. They then purchased another thermostat which was easily wired and did turn off when it was turned all the way to the left. It was installed with a five kilowatt heater in the garage and worked well, preventing a potential fire due to overheated connections. The customer would purchase again and the installation was done by a certified electrician.'